## Import required libraries

In [1]:
import pandas as pd
import tellurium as te
import sbmlnetwork
from Bio.KEGG.KGML import KGML_parser
from scipy.spatial.distance import euclidean

## Load and Parse Model and Metadata

- Load an SBML model from an Antimony string using Tellurium.
- Load KEGG pathway XML for coordinates to map onto
- Load a CSV containing labels to use
- Load the CSV used to build the Antimony file, detailing all reactions

In [ ]:
# Load Antimony model and convert to SBML
r = te.loada('SI_4_FullyAnnotatedAntimony_kegg.txt')
net = sbmlnetwork.load(r.getSBML())

# Load KEGG XML and label mapping
xml_file = 'ko01100.xml'
pathway = KGML_parser.read(open(xml_file, 'r'))
data = pd.read_csv('kegg_labels_add.csv', dtype='str', encoding='us-ascii', encoding_errors='ignore')
data = data.where(data.notnull(), None)

df = pd.read_csv('Reactions_M3_B_ordered.csv')

## Map KEGG Reactions to Graphical Entry IDs

Map KEGG reaction IDs to their corresponding graphical entry IDs using orthology annotations from the XML file.

In [4]:
# Dictionary mapping KEGG reaction IDs to ortholog group IDs
orthology_ids = {
    'R00315': ['K00925'], 'R01353': ['K00925', 'K00932', 'K19697'],
    'R00235': ['K01895', 'K01913'], 'R00236': ['K01895', 'K01913'], 'R00316': ['K01895', 'K01913'],
    'R00925': ['K01895', 'K01908'],
    'R00742': ['K01946','K01961','K01962','K01963','K01964','K02160','K11262','K11263','K15036','K15037','K18472','K18603','K18604','K18605','K19312','K22568'],
    'R01324': ['K01681','K01682','K27802'], 'R01325': ['K01681','K01682','K27802'], 'R01900': ['K01681','K01682','K27802'],
    'R00754': ['K00001','K00121','K04022','K04072','K13951','K13952','K13953','K13954','K13980','K18857'],
    'R00746': ['K00002','K12957','K13979'], 'R00352': ['K01648','K15230','K15231'],
    'R00351': ['K01647','K01659','K05942','K27797'],
    'R03815': ['K00382'], 'R00209': ['K00161','K00162','K00163','K00382','K00627'],
    'R01221': ['K00281','K00282','K00283','K00382','K00605','K02437'],
    'R01933': ['K00382','K00658','K15791'], 'R08549': ['K00164','K00382','K00658','K01616'],
    'R00704': ['K03777','K03778'],
    'R01082': ['K01675','K01676','K01677','K01678','K01679','K01774'],
    'R01736': ['K01069'],
    'R00267': ['K00031'], 'R00268': ['K00031'], 'R01899': ['K00031'],
    'R00342': ['K00024','K00025','K00026'], 'R00214': ['K00027','K00028'],
    'R00217': ['K01003'], 'R00216': ['K00029'],
    'R00230': ['K00625','K04020','K13788','K15024'],
    'R00921': ['K00625','K13788','K13923','K15024'],
    'R00341': ['K01610'], 'R00345': ['K01595'],
    'R00200': ['K00873','K12406'], 'R00199': ['K01007'],
    'R02164': ['K00233','K00234','K00235','K00236','K00237','K00239','K00240','K00241','K00242','K00244','K00245','K00246','K00247','K18859','K18860','K25801','K25995','K25996'],
    'R00405': ['K01902','K01903'], 'R02404': ['K01902','K01903'],
    'R00220': ['K01752','K01754','K17989'],
    'R00519': ['K00122','K00123','K00124','K00126','K00127','K22515'],
    'R00344': ['K01958','K01959','K01960'],
    'R00226': ['K01652','K01653','K11258'], 'R08648': ['K01652','K01653','K11258'],
    'R00945': ['K00600'], 'R09099': ['K00600'],
    'R00497': ['K01920','K21456'], 'R00371': ['K00639'],
    'R03425': ['K00281','K00282','K00283'], 'R00479': ['K01637']
}

In [5]:
def map_reaction_to_entry_id(xml_file_path, reaction_orthology_dict):
    reaction_entry_map = {}
    
    for entry in pathway.entries.values():
        if entry.type != "ortholog":
            continue

        orthology_ids_in_entry = set(entry.name.replace('ko:', '').split())

        for reaction_id, orthologs in reaction_orthology_dict.items():
            if set(orthologs) & orthology_ids_in_entry:
                reaction_entry_map.setdefault(reaction_id, []).append(entry.id)
    
    return reaction_entry_map

reaction_dict = map_reaction_to_entry_id(xml_file, orthology_ids)

## Extract (x, y) Coordinates of Substrates and Products

- Extract graphical positions for compounds participating in reactions.
- Store positions keyed by compound ID, reaction ID, and entry ID.

In [6]:
def extract_cpd_ids(rxn_str, label):
    """Extract cleaned compound IDs (without 'cpd:') from a reaction string under a given label."""
    try:
        line = next(l for l in rxn_str.splitlines() if label in l)
        part = line.split(f'{label}: ')[1]
        return [s.strip().replace('cpd:', '') for s in part.split(',')]
    except (StopIteration, IndexError):
        return []

def get_entry_coords(entry_id):
    """Safely return (x, y) coordinates of a pathway entry."""
    entry = pathway.entries.get(entry_id)
    if entry and entry.graphics:
        return entry.graphics[0].x, entry.graphics[0].y
    return None

# Load the pathway and initialize the map
cpd_coordinates_map = {}

for reaction_id, entry_ids in reaction_dict.items():
    for entry_id in entry_ids:
        reaction_entry = next((r for r in pathway.reactions if r.id == entry_id), None)
        if not reaction_entry:
            continue

        if reaction_id not in [rn.replace('rn:', '') for rn in reaction_entry._names]:
            continue

        rxn_str = str(reaction_entry)
        substrate_cpds = extract_cpd_ids(rxn_str, 'Substrates')
        product_cpds = extract_cpd_ids(rxn_str, 'Products')

        for i, sid in enumerate(reaction_entry._substrates):
            if i < len(substrate_cpds):
                coords = get_entry_coords(sid)
                if coords:
                    key = f"{substrate_cpds[i]}_{reaction_id}_{entry_id}"
                    cpd_coordinates_map[key] = coords

        for i, pid in enumerate(reaction_entry._products):
            if i < len(product_cpds):
                coords = get_entry_coords(pid)
                if coords:
                    key = f"{product_cpds[i]}_{reaction_id}_{entry_id}"
                    cpd_coordinates_map[key] = coords

## Translate Compound IDs Using Custom Label Mapping

Replace KEGG compound IDs in the coordinate map with custom labels based on a CSV mapping.

In [7]:
cpd_coordinates_map_tran = {}
for key, value in cpd_coordinates_map.items():
    new_key = key
    for _, row in data.iterrows():
        if row['KEGG ID'] in new_key:
            new_key = new_key.replace(row['KEGG ID'], row['ID'])
    cpd_coordinates_map_tran[new_key] = value

## Apply KEGG coordinates to SBMLNetwork layout

Step 1: Auto-layout the network to initialize positions

In [8]:
net.auto_layout(max_num_connected_edges=1000)

Step 2: Load KEGG reaction mappings

In [ ]:
label_to_reaction_id = dict(zip(df['Label'], df['Reaction ID']))
reaction_id_to_label = {v: k for k, v in label_to_reaction_id.items()}

Step 3: Get current reaction IDs in network and map to KEGG IDs

In [10]:
net_reactions = net.get_reactions_list()
M3_reaction_labels = net.get_reaction_ids()
M3_reactionIDs = [label_to_reaction_id.get(label, label) for label in M3_reaction_labels]

# Build label-to-reaction object dictionary
label_to_ID_dict = {
    reaction.get_reaction_id(): {'id': M3_reactionIDs[i], 'reaction_object': reaction}
    for i, reaction in enumerate(net_reactions)
}

Step 4: Translate KEGG-mapped coordinates

In [11]:
translated_cpd_coordinates = {}
for key, coords in cpd_coordinates_map_tran.items():
    species, reaction, entry_id = key.rsplit("_", 2)
    try:
        reaction_label = reaction_id_to_label[reaction]
        size = net.get_species(species).get_size()
        updated_coords = (coords[0] - size[0]/2, coords[1] - size[1]/2)
        translated_cpd_coordinates[(species, reaction_label, entry_id)] = updated_coords
    except KeyError:
        continue

Step 5: Organize coordinate data for species/reactions/entries

In [12]:
species_mapping = {}
reaction_mapping = {}
entry_mapping = {}

for (species, reaction, entry_id), coords in translated_cpd_coordinates.items():
    species_mapping.setdefault(species, {}).setdefault(reaction, {})[entry_id] = coords
    reaction_mapping.setdefault(reaction, {}).setdefault(species, {})[entry_id] = coords
    entry_mapping.setdefault(entry_id, {}).setdefault(reaction, []).append(species)

Step 6: Assign coordinates or handle aliases

In [13]:
set_coordinates = {}
reaction_aliases = {}
multipleentryids = []
special_metabs = []

for spc in net.get_species_list():
    try:
        allcoords = species_mapping[spc.get_species_id()]
    except KeyError:
        spc.hide()
        continue

    # Determine if coordinates are all identical
    coordinates = {coord for subdict in allcoords.values() for coord in subdict.values()}
    all_same = len(coordinates) == 1

    if all_same:
        coord = next(iter(coordinates))
        if coord not in set_coordinates:
            spc.set_position(coord)
            set_coordinates[coord] = spc.get_species_id()
    else:
        special_metabs.append(spc.get_species_id())
        for reaction, entry_coords in allcoords.items():
            if len(entry_coords) > 1 or len(allcoords) > 1:
                multipleentryids.append(reaction)

# Identify duplicate reactions requiring aliases
duplicatereactions = list({
    rxn for rxn in multipleentryids
    for v in reaction_mapping[rxn].values()
    if len(v) > 1
})

Step 7: Manually set positions and create aliases for specific reactions

In [14]:
for metabolite, entry_coords in reaction_mapping['R64'].items():
    entry_ids = list(entry_coords.keys())  # Get keys as a list

id_index_assignments = {}

# R64 setup
for spc in net.get_reaction('R64').get_species_list():
    if spc.get_species_id() in reaction_mapping['R64'].keys():
        spc.set_position(reaction_mapping['R64'][spc.get_species_id()][entry_ids[0]])
        id_index_assignments[entry_ids[0]] = 0
    else:
        spc.hide()

aliasreaction = net.get_reaction('R64').create_alias()

for spc in aliasreaction.get_species_list():
    if spc.get_species_id() in reaction_mapping['R64'].keys():
        if spc.get_species_id() == 'AcCoA':
            aliasreaction.assign_species(net.get_species_list('AcCoA')[0])
        else:
            spc.set_position(reaction_mapping['R64'][spc.get_species_id()][entry_ids[1]])
            id_index_assignments[entry_ids[1]] = 1
    else:
        spc.hide()

# R227 setup
net.get_reactions_list('R227').get_species_list()[0].set_position(reaction_mapping['R227']['Serine']['7890']) # Serine
net.get_reactions_list('R227').get_species_list()[1].set_position(reaction_mapping['R227']['THF']['7890']) # THF
net.get_reactions_list('R227').get_species_list()[2].hide() # enzyme
net.get_reactions_list('R227').get_species_list()[3].hide() # water
net.get_reactions_list('R227').get_species_list()[4].set_position(reaction_mapping['R227']['Glycine']['7890']) # Glycine, 7890 is the entry ID for index = 0 reaction
net.get_reactions_list('R227').get_species_list()[5].set_position(reaction_mapping['R227']['_510CH2THF']['7890']) # 510CH2THF

alias227 = net.get_reactions_list('R227').create_alias()

alias227[0].assign_species(net.get_reactions_list('R227').get_species_list()[3])
alias227.get_species_list()[0].hide()
alias227.get_species_list()[5].set_position(reaction_mapping['R227']['_510CH2THF']['5411'])
alias227.get_species_list()[2].hide()
alias227.get_species_list()[4].hide()
alias227.get_species_list()[1].set_position(reaction_mapping['R227']['THF']['5411'])

# R269 setup
net.get_reactions_list('R269').get_species_list()

alias_H = net.get_species("H").create_alias(net.get_reactions_list('R269')[0])
alias_H.hide()

net.get_reactions_list('R269').get_species_list()[0].hide()
net.get_reactions_list('R269').get_species_list()[1].set_position(reaction_mapping['R269']['Oxa']['4357'])
net.get_reactions_list('R269')[0].assign_species(alias_H)

net.get_reactions_list('R269').get_species_list()[3].hide()
net.get_reactions_list('R269').get_species_list()[4].hide()
net.get_reactions_list('R269').get_species_list()[5].set_position(reaction_mapping['R269']['Mal']['4357'])

alias269 = net.get_reactions_list('R269').create_alias()

alias269.get_species_list()[3].hide()
alias269.get_species_list()[5].set_position(reaction_mapping['R269']['Mal']['5622'])
alias269.get_species_list()[4].hide()
alias269.get_species_list()[0].hide()
alias269.get_species_list()[1].set_position(reaction_mapping['R269']['Oxa']['5622'])
alias269[0].assign_species(alias_H)

# R308 setup
net.get_reactions_list('R308').get_species_list()

net.get_reactions_list('R308').get_species_list()[4].hide()
net.get_reactions_list('R308')[0].assign_species(net.get_reactions_list('R269').get_species_list()[1])
net.get_reactions_list('R308').get_species_list()[3].hide()
net.get_reactions_list('R308').get_species_list()[0].hide()
net.get_reactions_list('R308').get_species_list()[1].hide()
net.get_reactions_list('R308').get_species_list()[2].set_position(reaction_mapping['R308']['PEP']['3007'])

alias308 = net.get_reactions_list('R308').create_alias()

alias308.get_species_list()[4].hide()
alias308[0].assign_species(alias269.get_species_list()[1])
alias308.get_species_list()[3].hide()
alias308.get_species_list()[0].hide()
alias308[0].assign_species(net.get_reactions_list('R308').get_species_list()[3])
alias308.get_species_list()[1].set_position(reaction_mapping['R308']['CO2']['2412'])
alias308.get_species_list()[2].set_position(reaction_mapping['R308']['PEP']['2412'])

# R307 setup
net.get_reactions_list('R307').get_species_list()[4].hide()
net.get_reactions_list('R307')[0].assign_species(net.get_reactions_list('R269').get_species_list()[1])
net.get_reactions_list('R307').get_species_list()[3].hide()
net.get_reactions_list('R307').get_species_list()[0].hide()
net.get_reactions_list('R307').get_species_list()[1].hide()
net.get_reactions_list('R307')[0].assign_species(net.get_reactions_list('R308').get_species_list()[2])

alias307 = net.get_reactions_list('R307').create_alias()

alias307.get_species_list()[4].hide()
alias307[0].assign_species(alias269.get_species_list()[1])
alias307.get_species_list()[3].hide()
alias307.get_species_list()[0].hide()
alias307.get_species_list()[1].hide()
alias307[0].assign_species(alias308.get_species_list()[2])

# Pyr setup for R270 and R272
net.get_species('Pyr').set_position(reaction_mapping['R346']['Pyr']['1556'])
j1_reaction = net.get_reaction("R270")
alias_species = net.get_species("Pyr").create_alias(j1_reaction)
alias_species.set_position(reaction_mapping['R270']['Pyr']['2409'])

j2_reaction = net.get_reaction("R272")
j2_reaction.assign_species(alias_species)

True

## Additional aesthetic choices:
### Leveraging reaction arrow colors to denote fluxes

In [15]:
# net.show_fluxes(10, log_scale=True)

### Grouping biosynthetic modules visually with color

In [16]:
# pathway_dict = {
#     'Redox & Energy Metabolism': [
#         'R83',  # Alcohol dehydrogenase
#         'R90',  # Alcohol dehydrogenase (nadp+)
#         'R181',  # D-lactate dehydrogenase
#         'R197',  # Formate dehydrogenase
#         'R169', 'R171', 'R172', 'R173', 'R174',  # Dihydrolipoyl dehydrogenase
#         'R175',  # Dihydrolipoyllysine-residue acetyltransferase
#         'R176', 'R177'  # Dihydrolipoyllysine-residue succinyltransferase
#     ],
#     'Amino Acid Metabolism': [
#         'R406',  # L-serine ammonia-lyase
#         'R62', 'R63',  # Acetolactate synthase
#         'R227', 'R228',  # Glycine hydroxymethyltransferase
#         'R224',  # Glycine c-acetyltransferase
#         'R225',  # Glycine dehydrogenase (aminomethyl-transferring)
#         'R219'  # Glutathione synthase
#     ],
#     'Pyruvate Metabolism & Acetyl-CoA Formation': [
#         'R346',  # Pyruvate dehydrogenase (acetyl-transferring)
#         'R345',  # Pyruvate carboxylase
#         'R56', 'R57',  # Acetate kinase
#         'R58', 'R59', 'R60', 'R61',  # Acetate-coa ligase
#         'R64',  # Acetyl-coa carboxylase
#         'R304', 'R305',  # Phosphate acetyltransferase
#         'R245',  # Hydroxyacylglutathione hydrolase
#         'R308'  # Phosphoenolpyruvate carboxylase
#     ],
#     'Glycolysis / Gluconeogenesis': [
#         'R347',  # Pyruvate kinase
#         'R351',  # Pyruvate, water dikinase
#         'R307'   # Phosphoenolpyruvate carboxykinase (atp)
#     ],
#     'TCA Cycle': [
#         'R157',  # Citrate synthase
#         'R69', 'R70', 'R71',  # Aconitate hydratase
#         'R255', 'R256', 'R257',  # Isocitrate dehydrogenase (nadp+)
#         'R258',  # Isocitrate lyase
#         'R299', 'R300',  # Oxoglutarate dehydrogenase (succinyl-transferring)
#         'R268', 'R269',  # Malate dehydrogenase
#         'R270', 'R271', 'R272', 'R273',  # Malate dehydrogenase (oxaloacetate-decarboxylating)
#         'R204',  # Fumarate hydratase
#         'R371',  # Succinate dehydrogenase
#         'R372', 'R373',  # Succinate-coa ligase (adp-forming)
#         'R123',  # Atp citrate synthase
#     ]
# }

In [17]:
# colors = ['#7a0606', '#07075c', '#ad6309', '#6c0d8f', '#15661f']
# for i, mod in enumerate(pathway_dict.keys()):
#     net.group_reactions(pathway_dict[mod], colors[i])

### Utilizing size of species nodes to convey concentrations

In [18]:
# r.reset()

# r.simulate(0,10)

# speciesids = r.getIds()[0:63]

# concentration_dict = {}
# for spc in speciesids:
#     if spc == 'H2O':
#         continue
#     elif spc == 'C00158HCT':
#         concentration_dict[spc] = r.getValue(spc)
#     else:
#         label = id_to_label[spc]
#         if label == 'H2O':
#             continue
#         concentration_dict[label] = r.getValue(spc)

# for key in concentration_dict:
#     if concentration_dict[key] < 0:
#         concentration_dict[key] = 0.0
#     concentration_dict[key] += 0.0001

# maxconc = max(concentration_dict.values())

# for key in concentration_dict:
#     concentration_dict[key] = concentration_dict[key]/maxconc

In [19]:
# net.show_concentrations(data=concentration_dict, show_by='size', log_scale=True, min_size=5, max_size=60)

## Setting reaction curves

In [20]:
# Hide all reactions initially
net.get_reactions_list().hide()

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [21]:
# Build a nested dictionary: reaction_mapping[reaction][entry_id][species] = coords
reaction_mapping = {}
for (species, reaction, entry_id), coords in translated_cpd_coordinates.items():
    reaction_mapping.setdefault(reaction, {}).setdefault(entry_id, {})[species] = coords

In [22]:
def get_species(entry_mapping, entry_id):
    """Return a unique list of species involved in a given entry ID."""
    species = set()
    if entry_id in entry_mapping:
        for reaction in entry_mapping[entry_id].values():
            species.update(reaction)
    return list(species)

def get_unique_coordinates(entry_mapping, species_mapping, entry_id):
    """Return ordered unique coordinates and species for a given entry ID."""
    species_list = get_species(entry_mapping, entry_id)
    coordinates = []
    seen = set()
    for species in species_list:
        if species in species_mapping:
            for reaction_data in species_mapping[species].values():
                if entry_id in reaction_data:
                    coord = reaction_data[entry_id]
                    if coord not in seen:
                        coordinates.append(coord)
                        seen.add(coord)
    return coordinates, species_list

In [23]:
def find_curve_origin(coords_list, reference_point):
    """Return the index of the point closest to the reference point."""
    distances = [euclidean(reference_point, p) for p in coords_list]
    return distances.index(min(distances))

In [24]:
species = net.get_species_list()

for s in species:
    s.set_shape('circle')
    s.set_size([30, 30])
    spcname = s.get_species_id()
    for _, row in data.iterrows():
        if spcname == row['ID']:
            s.set_text(row['Label'])

species.move_texts_by((30, 30))

[[True],
 [True],
 [True],
 [True],
 [True, True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [Tr

In [25]:
id_to_label = pd.Series(data['ID'].values, index=data['KEGG ID']).to_dict()

for reaction, entries in reaction_mapping.items():
    for i, entry_id in enumerate(entries):

        for graphicsid in range(len(pathway.entries[int(entry_id)].graphics)):
            linecoords = pathway.entries[int(entry_id)].graphics[graphicsid].coords
            rxn = net.get_reactions_list(reaction)[i]

            # Set reaction node position
            mid_point = ((linecoords[0][0] + linecoords[-1][0]) / 2, 
                         (linecoords[0][1] + linecoords[-1][1]) / 2)
            rxn.set_position(mid_point)

            first_point, last_point = linecoords[0], linecoords[-1]
            involved_coords, involved_species = get_unique_coordinates(entry_mapping, species_mapping, entry_id)
            start_idx = find_curve_origin(involved_coords, first_point)
            end_idx = find_curve_origin(involved_coords, last_point)

            # Determine middle point for breaking
            middle_point = (linecoords[len(linecoords)//2] 
                            if len(linecoords) > 2 
                            else mid_point)

            for j, species_idx in enumerate([start_idx, end_idx]):
                species_id = involved_species[species_idx]
                curve = rxn.get_curves_list(rxn.get_species(species_id)[0])[0]
                curve.show()

                terminate = False
                current_index = 0

                # Define buffer zones around species coordinates
                coverage_regions = [
                    (coord[0] - 18, coord[0] + 18, coord[1] - 18, coord[1] + 18) 
                    for coord in involved_coords
                ]

                if j == 1:
                    linecoords.reverse()

                for k in range(len(linecoords) - 1):
                    if terminate:
                        break

                    start, end = linecoords[k], linecoords[k + 1]
                    if start == middle_point:
                        break

                    for x_min, x_max, y_min, y_max in coverage_regions:
                        if ((x_min <= end[0] <= x_max and y_min <= end[1] <= y_max) and
                            not (x_min < start[0] < x_max and y_min < start[1] < y_max)):
                            
                            if not any(
                                x_min <= linecoords[-1][0] <= x_max and
                                y_min <= linecoords[-1][1] <= y_max
                                for x_min, x_max, y_min, y_max in coverage_regions):
                                curve.add_segment(start, end, start, end)
                                break

                            adjusted_x = x_max if start[0] >= x_max else x_min
                            adjusted_y = y_min if start[1] <= y_min else y_max

                            if current_index == 0:
                                curve.remove_segment(0)

                            curve.add_segment(start, (adjusted_x, adjusted_y), start, (adjusted_x, adjusted_y))
                            terminate = True
                            break

                    if not terminate:
                        if current_index == 0:
                            segment = curve.get_segment(current_index)
                            segment.set_start(start)
                            segment.set_end(end)
                            segment.set_control_point_1(start)
                            segment.set_control_point_2(end)
                            curve.add_segment(start, end, start, end)
                        else:
                            curve.add_segment(start, end, start, end)

                    current_index += 1

In [ ]:
net.compartment.set_size([5059.370, 3336.151])
net.compartment.set_fill_color('white')
net.set_size([5060.370, 3337.151], adjust_elements=False)

# Export to PDF
net.draw('MapDrawing.pdf', update_network_extents=False)

In [ ]:
net.save('M3Visualization.sbml')

True